# Try 16: setup `paprika` restraints

Presuming we have a host-guest system already setup, let's see if we can rationally instruct `paprika` to setup all the restraints for a real system.

In [1]:
%load_ext autoreload
%autoreload 2

import os as os
import numpy as np

import parmed as pmd
import pytraj as pt

In [2]:
import logging
import datetime as dt
d_date = dt.datetime.now()
logging.basicConfig(filename='paprika' + '-' + d_date.strftime("%Y-%m-%d-%I-%M-%S-%p") + '.log', 
                    format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %I:%M:%S %p',
                    level=logging.DEBUG)
logging.info('Started logging...')

import paprika
print(paprika.__version__)

from paprika.restraints import static_DAT_restraint
from paprika.restraints import DAT_restraint
from paprika.restraints import amber_restraint_line
from paprika.restraints import create_window_list

from paprika.utils import make_window_dirs

2018-04-02_15:06:19_-0700-5a6c2cb-0.0.3


## Preamble

We are going to setup host translational restraints, guest translational restraints, host "jacks," and guest wall restraints with `paprika`. First, let's try working in a directory where we have a known `disang.rest` file we can use as a reference: `/data/davids4/projects/smirnoff-host-guest/a-bam-p/a00/original` or `/data/davids4/projects/smirnoff-host-guest/a-bam-p/u00/original` for the final values.

<img src="images/paprika.png?2">

In [3]:
dummy_anchors = [':8', ':9', ':10']
host_anchors  = [':1@O3', ':3@C1', ':5@C6']
guest_anchors = [':7@C4', ':7@N1']

In [4]:
hg = pmd.load_file('systems/a-bam-p/smirnoff/a000/smirnoff.prmtop',
                   'systems/a-bam-p/smirnoff/a000/smirnoff.inpcrd',
                    structure=True)

The attach fractions can be found in the `Setup.pl` file in the original location on `kirkwood`: `/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/a-bam-p/a00/` under `@AttachFC`.

The attach force constant is 5.0 kcal/mol-A^{2} for distances and 100.0 kcal/mol-rad^{2} for angels and torsions based on `disang.rest` in `u00`.

The pull distances are specified by `@Translate` in `Setup.pl` and are offset by 6 Angstroms.

In [5]:
attach_string = '0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 18.10 24.40 37.00 49.60 74.80 100.00'
attach_fractions = [float(i) / 100 for i in attach_string.split()]

pull_string = '0.00 0.40 0.80 1.20 1.60 2.00 2.40 2.80 3.20 3.60 4.00 4.40 4.80 5.20 5.60 6.00 6.40 6.80 7.20 7.60 8.00 8.40 8.80 9.20 9.60 10.00 10.40 10.80 11.20 11.60 12.00 12.40 12.80 13.20 13.60 14.00 14.40 14.80 15.20 15.60 16.00 16.40 16.80 17.20 17.60 18.00'
pull_distances = [float(i) + 6.00 for i in pull_string.split()]

windows = [len(attach_fractions), len(pull_distances), 0]
print(f'There are {windows} windows in this attach-pull calculation.')

There are [15, 46, 0] windows in this attach-pull calculation.


## Define the host translational restraints
These restraints help pin down the host during initial attachment of the DAT restraints. They are static, meaning unlike other restraints, they are not turned on slowly during the attachment phase.

In [6]:
static_restraint_atoms = [[dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[2], dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1], host_anchors[2]],
                        ]

static_restraint_distance_fc = 5.0
static_restraint_angle_fc = 100.0

## Define the guest translational restraints

In [7]:
guest_restraint_atoms = [[dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[1], dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[0], guest_anchors[0], guest_anchors[1]],
                        ]

guest_restraint_targets = [6.0,
                           180.0,
                           180.0
                          ]
guest_restraint_target_final = [24.0, 180.0, 180.0]
guest_restraint_distance_fc = 5.0
guest_restraint_angle_fc = 100.0

## Define the host conformational restraints

The "jacks" restraints can be written O5_n - C1_n - O1_n - C4_n+1 and C1_n - O1_n - C4_n+1 - C5_n+1.

In [8]:
host_conformational_template = [['O5', 'C1', 'O1', 'C4'],
                                ['C1', 'O1', 'C4', 'C5']
                               ]

host_residues = len(hg[':MGO'].residues)
first_host_residue = hg[':MGO'].residues[0].number + 1
conformational_restraint_atoms = []
conformational_restraint_targets = []
conformational_restraint_fc = 6.0

for n in range(first_host_residue, host_residues + first_host_residue):
    if n + 1 < host_residues + first_host_residue:
        next_residue = n + 1
    else:
        next_residue = first_host_residue
    conformational_restraint_atoms.append(
        [f':{n}@{host_conformational_template[0][0]}',
         f':{n}@{host_conformational_template[0][1]}',
         f':{n}@{host_conformational_template[0][2]}',
         f':{next_residue}@{host_conformational_template[0][3]}'
        ])
    conformational_restraint_targets.append(104.30)
    conformational_restraint_atoms.append(
        [f':{n}@{host_conformational_template[1][0]}',
         f':{n}@{host_conformational_template[1][1]}',
         f':{next_residue}@{host_conformational_template[1][2]}',
         f':{next_residue}@{host_conformational_template[1][3]}'
        ])
    conformational_restraint_targets.append(-108.8)

## Define the guest "wall" restraints

The guest wall restraints are only on during the attachment phase, and make sure that the guest does not unbind and waste time diffusing around the simulation box. These distance restraints are flat from zero through the restraint target value, near the edge of the binding cavity, and then increase harmonically beyond that. The guest "wall" angle restraint is harmonic and prevents the guest from flipping around during the bound phase.

In [9]:
guest_wall_template = [
    ['O2', guest_anchors[0]],
    ['O6', guest_anchors[0]]
]

guest_wall_restraint_atoms = []
guest_wall_restraint_targets = []
guest_wall_restraint_angle_fc = 500.0
guest_wall_restraint_distance_fc = 50.0

for n in range(first_host_residue, host_residues + first_host_residue):
    guest_wall_restraint_atoms.append(
        [f':{n}@{guest_wall_template[0][0]}',
         f'{guest_wall_template[0][1]}',
        ])
    guest_wall_restraint_targets.append(11.3)
    guest_wall_restraint_atoms.append(
        [f':{n}@{guest_wall_template[1][0]}',
         f'{guest_wall_template[1][1]}',
        ])
    guest_wall_restraint_targets.append(13.3)

guest_wall_restraint_atoms.append([dummy_anchors[1],
                                 guest_anchors[0],
                                 guest_anchors[1]])
guest_wall_restraint_targets.append(80.0)

## Initialize all the restraints

In [10]:
for index, atoms in enumerate(static_restraint_atoms):    
    static_DAT_restraint(restraint_mask_list=atoms,
                        num_window_list=windows, 
                        ref_structure=hg,
                        force_constant=static_restraint_angle_fc if len(atoms) > 2 else static_restraint_distance_fc,
                        amber_index=True)

In [11]:
for index, atoms in enumerate(guest_restraint_atoms): 
    if len(atoms) > 2:
        angle = True
    else:
        angle = False
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    if angle:
        this.mask3 = atoms[2]
        this.attach['fc_final'] = guest_restraint_angle_fc
    else:
        this.attach['fc_final'] = guest_restraint_distance_fc
    this.attach['target'] = guest_restraint_targets[index]
    this.attach['fraction_list'] = attach_fractions
    
    
    this.pull['target_final'] = guest_restraint_target_final[index]
    this.pull['num_windows'] = windows[1]
    this.initialize()
    
    print(amber_restraint_line(this, 'pull', 10))

&rst iat= 144,140,          r1=    0.00000, r2=   10.00000, r3=   10.00000, r4=  999.00000, rk2=    5.00000, rk3=    5.00000,  &end

&rst iat= 145,144,140,      r1=    0.00000, r2=  180.00000, r3=  180.00000, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 144,140,127,      r1=    0.00000, r2=  180.00000, r3=  180.00000, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end



In [12]:
for index, atoms in enumerate(conformational_restraint_atoms): 
    
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    this.mask3 = atoms[2]
    this.mask4 = atoms[3]
    
    this.attach['fraction_list'] = attach_fractions
    this.attach['target'] = conformational_restraint_targets[index]
    this.attach['fc_final'] = conformational_restraint_fc
    this.pull['target_final'] = conformational_restraint_targets[index]
    this.pull['num_windows'] = windows[1]
    
    this.initialize()

In [13]:
for index, atoms in enumerate(guest_wall_restraint_atoms): 
    if len(atoms) > 2:
        angle = True
    else:
        angle = False
    
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    if angle:
        this.mask3 = atoms[2]
        this.attach['fc_initial'] = guest_wall_restraint_angle_fc
        this.attach['fc_final'] = guest_wall_restraint_angle_fc
        this.custom_restraint_values['rk2'] = 500.0
        this.custom_restraint_values['rk3'] = 0.0
    else:
        this.attach['fc_initial'] = guest_wall_restraint_distance_fc
        this.attach['fc_final'] = guest_wall_restraint_distance_fc
        this.custom_restraint_values['rk2'] = 50.0
        this.custom_restraint_values['rk3'] = 50.0
        this.custom_restraint_values['r1'] = 0.0
        this.custom_restraint_values['r2'] = 0.0

    this.attach['target'] = guest_wall_restraint_targets[index]
    this.attach['num_windows'] = len(attach_fractions)
    
    this.initialize()

In [14]:
DAT_restraint.instances[-2].phase

{'attach': {'force_constants': array([ 50.,  50.,  50., ...,  50.,  50.,  50.]),
  'targets': array([ 13.3,  13.3,  13.3, ...,  13.3,  13.3,  13.3])},
 'pull': {'force_constants': None, 'targets': None},
 'release': {'force_constants': None, 'targets': None}}

## Make the window list

In [18]:
window_list = create_window_list(DAT_restraint.instances)
print(window_list)

['a000', 'a001', 'a002', 'a003', 'a004', 'a005', 'a006', 'a007', 'a008', 'a009', 'a010', 'a011', 'a012', 'a013', 'p000', 'p001', 'p002', 'p003', 'p004', 'p005', 'p006', 'p007', 'p008', 'p009', 'p010', 'p011', 'p012', 'p013', 'p014', 'p015', 'p016', 'p017', 'p018', 'p019', 'p020', 'p021', 'p022', 'p023', 'p024', 'p025', 'p026', 'p027', 'p028', 'p029', 'p030', 'p031', 'p032', 'p033', 'p034', 'p035', 'p036', 'p037', 'p038', 'p039', 'p040', 'p041', 'p042', 'p043', 'p044', 'p045']


In [15]:
# make_window_dirs(window_list, path='/systems/a-bam-p/smirnoff/')

In [17]:
for window in window_list:
    with open('systems/a-bam-p/smirnoff/' + window + '/disang.rest', 'w') as file:
        for restraint in DAT_restraint.instances:
            if window[0] == 'a':
                phase = 'attach'
            if window[0] == 'p':
                phase = 'pull'
            if window[0] == 'r':
                phase = 'release'
            # print(amber_restraint_line(restraint, phase, int(window[1:])))
            string = amber_restraint_line(restraint, phase, int(window[1:]))
            print(string)
            if string is not None:
                file.write(string)

&rst iat= 144,10,           r1=    0.00000, r2=    7.49023, r3=    7.49023, r4=  999.00000, rk2=    5.00000, rk3=    5.00000,  &end

&rst iat= 145,144,10,       r1=    0.00000, r2=  131.36554, r3=  131.36554, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 146,145,144,10,   r1=  -64.88460, r2=  115.11540, r3=  115.11540, r4=  295.11540, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 144,10,43,        r1=    0.00000, r2=   55.11101, r3=   55.11101, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 145,144,10,43,    r1=  -15.40894, r2=  164.59106, r3=  164.59106, r4=  344.59106, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 144,10,43,101,    r1=  -70.84633, r2=  109.15367, r3=  109.15367, r4=  289.15367, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 144,140,          r1=    0.00000, r2=    6.00000, r3=    6.00000, r4=  999.00000, rk2=    0.00000, rk3=    0.00000,  &end

&rst iat= 145,144,140,      r1=    0.00000, r2=  180.00000, r3=  180.

None
None
None
None
None
None
None
&rst iat= 144,10,           r1=    0.00000, r2=    7.49023, r3=    7.49023, r4=  999.00000, rk2=    5.00000, rk3=    5.00000,  &end

&rst iat= 145,144,10,       r1=    0.00000, r2=  131.36554, r3=  131.36554, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 146,145,144,10,   r1=  -64.88460, r2=  115.11540, r3=  115.11540, r4=  295.11540, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 144,10,43,        r1=    0.00000, r2=   55.11101, r3=   55.11101, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 145,144,10,43,    r1=  -15.40894, r2=  164.59106, r3=  164.59106, r4=  344.59106, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 144,10,43,101,    r1=  -70.84633, r2=  109.15367, r3=  109.15367, r4=  289.15367, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 144,140,          r1=    0.00000, r2=   16.80000, r3=   16.80000, r4=  999.00000, rk2=    5.00000, rk3=    5.00000,  &end

&rst iat= 145,144,140,      r1=   

In [18]:
import json

In [19]:
DAT_restraint.instances[0]

In [21]:
DAT_restraint.instances[0].__dict__

{'amber_index': True,
 'attach': {'fc_final': 5.0,
  'fc_increment': None,
  'fc_initial': 5.0,
  'fc_list': None,
  'fraction_increment': None,
  'fraction_list': None,
  'num_windows': 15,
  'target': 7.4902344887461156},
 'auto_apr': False,
 'continuous_apr': True,
 'custom_restraint_values': {},
 'group1': False,
 'group2': False,
 'group3': False,
 'group4': False,
 'index1': [144],
 'index2': [10],
 'index3': None,
 'index4': None,
 'mask1': ':8',
 'mask2': ':1@O3',
 'mask3': None,
 'mask4': None,
 'phase': {'attach': {'force_constants': array([ 5.,  5.,  5., ...,  5.,  5.,  5.]),
   'targets': array([ 7.49023449,  7.49023449,  7.49023449, ...,  7.49023449,
           7.49023449,  7.49023449])},
  'pull': {'force_constants': array([ 5.,  5.,  5., ...,  5.,  5.,  5.]),
   'targets': array([ 7.49023449,  7.49023449,  7.49023449, ...,  7.49023449,
           7.49023449,  7.49023449])},
  'release': {'force_constants': None, 'targets': None}},
 'pull': {'fc': 5.0,
  'fraction_increme

In [22]:
json.dumps(DAT_restraint.instances[0].__dict__)

TypeError: Object of type 'AmberParm' is not JSON serializable

In [27]:
def serialize(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, pmd.amber.AmberParm):
        serial = obj.__dict__
        return serial

    return obj.__dict__

In [29]:
json.dumps(DAT_restraint.instances[0], default=serialize)

AttributeError: 'mappingproxy' object has no attribute '__dict__'

In [36]:
from json import JSONEncoder

In [40]:
class MyEncoder(JSONEncoder):
    def default(self, obj):
        if not isinstance(obj, pmd.amber.AmberParm):
            return obj.__dict__
        else:
            return obj

In [42]:
MyEncoder().encode(DAT_restraint.instances[0].__dict__)

IndexError: list index out of range

In [54]:
DAT_restraint.instances

[]

In [ ]:
%xmode verbose

# Test analysis

In [27]:
simulation_data = {
    'attach'  : [],
    'pull'    : [],
    'release' : []
}

for window in window_list:
    phase_prefix = window[0]
    
    if phase_prefix == 'a':
        phase = 'attach'
    if phase_prefix == 'p':
        phase = 'pull'
    if phase_prefix == 'r':
        phase = 'release'
        
    window_index = int(window[1:])
        
    simulation_data[phase].append([])
    #simulation_data[phase][window_index] = []
        
    # for restraint_index, restraint in enumerate(DAT_restraint.instances[6]):
    
    for restraint_index, restraint in enumerate(DAT_restraint.instances[:-13]):

        simulation_data[phase][window_index].append([]) 
        # simulation_data[phase][window_index][restraint_index] = []

        structure = pt.load(os.path.join('systems/a-bam-p/smirnoff/', window, 'smirnoff.inpcrd'), 
                           os.path.join('systems/a-bam-p/smirnoff/', window, 'smirnoff.prmtop'))

        stripped = structure.strip(':WAT,:Na+,:Cl-')

        traj = pt.iterload(os.path.join('systems/a-bam-p/smirnoff', window, 'traj.nc'),
                          top=stripped.topology)

        if restraint.mask1 and restraint.mask2 and \
        not restraint.mask3 and not restraint.mask4:
            data = pt.distance(traj, ' '.join([restraint.mask1, restraint.mask2]))
        elif restraint.mask1 and restraint.mask2 and \
        restraint.mask3 and not restraint.mask4:
            data = pt.angle(traj, ' '.join([restraint.mask1, restraint.mask2, restraint.mask3]))
        elif restraint.mask1 and restraint.mask2 and \
        restraint.mask3 and restraint.mask4:
            data = pt.dihedral(traj, ' '.join([restraint.mask1, restraint.mask2, restraint.mask3, restraint.mask4]))

        simulation_data[phase][window_index][restraint_index] = data

In [28]:
simulation_data

{'attach': [[array([ 7.75383757,  7.5689578 ,  7.26134463, ...,  7.39523541,
           7.55594931,  7.46932635]),
   array([ 131.46999624,  129.86450208,  126.73574217, ...,  126.61190523,
           126.20105711,  126.22837444]),
   array([ 109.05187629,  113.49530429,  115.92386756, ...,  111.60676154,
           117.6869626 ,  110.05671112]),
   array([ 53.84142308,  55.01472458,  53.23366228, ...,  55.39381833,
           51.23943542,  50.672623  ]),
   array([ 168.62077504,  164.72789189,  168.88258782, ...,  162.71648501,
           163.64924024,  166.48649963]),
   array([ 100.90150179,  100.11506435,  105.34584179, ...,  110.94479079,
           102.20288163,  107.0472084 ]),
   array([ 4.40733076,  4.17582472,  3.59758547, ...,  3.79320455,
           3.42128891,  3.47231093]),
   array([ 167.35995862,  157.3969915 ,  177.68101008, ...,  140.66143879,
           134.59044228,  143.54663619]),
   array([ 140.17563263,  130.1179213 ,  164.49091301, ...,  112.70906418,
         

In [29]:
from paprika.analysis import fe_calc

In [30]:
analysis = fe_calc()

In [31]:
analysis.restraint_list = DAT_restraint.instances[:-13]

In [32]:
analysis.simulation_values = simulation_data

In [33]:
analysis.compute_free_energy()

!!!!!!!!!!!!!!
[False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
!!!!!!!!!!!!!!
[False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
!!!!!!!!!!!!!!
[False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
!!!!!!!!!!!!!!
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [52]:
%xmode verbose

Exception reporting mode: Verbose


In [ ]:
%pdb off

In [34]:
analysis.results

{'attach': {'mbar-block': {'convergence': array([ 0.05268182,  0.05268182,  0.02107661, ...,  0.02015721,
           0.02015721,  0.01609803]),
   'fe': 12.788795452886674,
   'fe_matrix': array([[  0.        ,   0.61239841,   1.04258791, ...,   9.70901502,
            11.46033165,  12.78879545],
          [ -0.61239841,   0.        ,   0.4301895 , ...,   9.0966166 ,
            10.84793324,  12.17639704],
          [ -1.04258791,  -0.4301895 ,   0.        , ...,   8.66642711,
            10.41774374,  11.74620754],
          ..., 
          [ -9.70901502,  -9.0966166 ,  -8.66642711, ...,   0.        ,
             1.75131664,   3.07978044],
          [-11.46033165, -10.84793324, -10.41774374, ...,  -1.75131664,
             0.        ,   1.3284638 ],
          [-12.78879545, -12.17639704, -11.74620754, ...,  -3.07978044,
            -1.3284638 ,   0.        ]]),
   'ordered_convergence': array([ 0.05268182,  0.05268182,  0.02107661, ...,  0.02015721,
           0.02015721,  0.01609803

In [26]:
analysis.results

{'attach': {'mbar-block': {'convergence': array([ 0.05268182,  0.05268182,  0.02107661, ...,  0.02015721,
           0.02015721,  0.01609803]),
   'fe': 12.78879545288669,
   'fe_matrix': array([[  0.        ,   0.61239841,   1.04258791, ...,   9.70901502,
            11.46033165,  12.78879545],
          [ -0.61239841,   0.        ,   0.4301895 , ...,   9.0966166 ,
            10.84793324,  12.17639704],
          [ -1.04258791,  -0.4301895 ,   0.        , ...,   8.66642711,
            10.41774374,  11.74620754],
          ..., 
          [ -9.70901502,  -9.0966166 ,  -8.66642711, ...,   0.        ,
             1.75131664,   3.07978044],
          [-11.46033165, -10.84793324, -10.41774374, ...,  -1.75131664,
             0.        ,   1.3284638 ],
          [-12.78879545, -12.17639704, -11.74620754, ...,  -3.07978044,
            -1.3284638 ,   0.        ]]),
   'ordered_convergence': array([ 0.05268182,  0.05268182,  0.02107661, ...,  0.02015721,
           0.02015721,  0.01609803]